In [1]:
import pandas as pd
import helpers
import json
import numpy as np
import random
import math

In [2]:
items = helpers.load_items_df()
items_dict = helpers.load_items()
domain_item_dict = {}
for item in items_dict.keys():
    domain = items_dict[item]['domain_id']
    if domain not in domain_item_dict:
        domain_item_dict[domain] = []
    domain_item_dict[domain].append(item)

In [3]:
interactions = helpers.load_interactions_df()

In [4]:
items_df = items[['item_id', 'domain_id', 'price', 'condition']].copy()

domains = items.domain_id.unique() 
m = int(math.log2(len(domains)) + 1)
columns = {f'domain_bit_{i}': [] for i in range(m)}
indexed_domains = {domains[i]: i for i in range(len(domains))}

def domain_apply(x):
    arr = helpers.bin_array(indexed_domains[x], m)
    for j in range(m):
        columns[f'domain_bit_{j}'].append(arr[j])

items_df['domain_id'].apply(domain_apply)
    
for k in columns.keys():
    items_df[k] = columns[k]
    
items_df['condition'] = items_df['condition'].apply(lambda x: 1 if x == 'new' else 0)
items_df = items_df.drop(columns=['domain_id'])
items_df

,item_id,price,condition,domain_bit_0,domain_bit_1,domain_bit_2,domain_bit_3,domain_bit_4,domain_bit_5,domain_bit_6,domain_bit_7,domain_bit_8,domain_bit_9,domain_bit_10,domain_bit_11,domain_bit_12
0,111260,1150000.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,871377,1392.83,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,490232,350.00,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1150706,3200.00,0,0,0,0,0,0,0,0,0,0,0,0,1,1
4,934912,1599.00,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102272,1099649,1600.00,0,0,1,0,0,1,1,1,0,1,1,0,1,1
2102273,1482187,69.90,1,0,0,1,1,1,0,0,0,0,0,1,1,1
2102274,1118904,118.00,1,0,1,0,0,1,1,1,1,1,0,0,0,0
2102275,237229,26.90,1,1,0,0,0,1,0,0,1,0,1,1,1,1


In [5]:
items_df

,item_id,price,condition,domain_bit_0,domain_bit_1,domain_bit_2,domain_bit_3,domain_bit_4,domain_bit_5,domain_bit_6,domain_bit_7,domain_bit_8,domain_bit_9,domain_bit_10,domain_bit_11,domain_bit_12
0,111260,1150000.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,871377,1392.83,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,490232,350.00,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1150706,3200.00,0,0,0,0,0,0,0,0,0,0,0,0,1,1
4,934912,1599.00,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102272,1099649,1600.00,0,0,1,0,0,1,1,1,0,1,1,0,1,1
2102273,1482187,69.90,1,0,0,1,1,1,0,0,0,0,0,1,1,1
2102274,1118904,118.00,1,0,1,0,0,1,1,1,1,1,0,0,0,0
2102275,237229,26.90,1,1,0,0,0,1,0,0,1,0,1,1,1,1


In [6]:
item_names_df = items[['item_id', 'title', 'category_id']]
item_names_df

,item_id,title,category_id
0,111260,Casa Sola En Venta Con Gran Patio Solo Pago De...,MLM170527
1,871377,Resident Evil Origins Collection Nintendo Swit...,MLM151595
2,490232,Falda De Imitación Piel Negra,MLM7697
3,1150706,Powercolor Red Devil Radeon Rx 580 8gb Gddr5,MLM9761
4,934912,Laptop Hp Nx6320 Core Duo Con Puerto Db9 Windo...,MLM1652
...,...,...,...
2102272,1099649,Carrinho De Bebê Stoke,MLB1386
2102273,1482187,Grelha Para Hambúrguer Preta Com Cabo Em Madei...,MLB193425
2102274,1118904,Meia Tam 7/8 Anti Embolia Trombose Antitrombo,MLB108791
2102275,237229,Pano De Boca Cremer Menina Luxo Bordado C/3 Und,MLB40629


In [7]:
users_df = pd.DataFrame()
users_df['user_id'] = interactions['user_id'].unique()
users_df

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
413158,413158
413159,413159
413160,413160
413161,413161


In [8]:
interactions_df = interactions[interactions['event_type'] != 'search'][['user_id', 'item_id', 'event_timestamp']]
interactions_df

,user_id,item_id,event_timestamp
0,0,1786148,2019-10-19T11:25:42.444-0400
1,0,1786148,2019-10-19T11:25:57.487-0400
3,0,1615991,2019-10-19T11:27:26.879-0400
4,0,1615991,2019-10-19T11:28:36.558-0400
5,0,1615991,2019-10-19T11:28:40.827-0400
...,...,...,...
11999161,413162,1348242,2019-10-04T08:34:56.361-0400
11999162,413162,792798,2019-10-07T10:49:32.217-0400
11999163,413162,258196,2019-10-07T10:52:21.825-0400
11999164,413162,12716,2019-10-07T10:53:07.399-0400


## Training

In [9]:
from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

#### Input data analysis

In [10]:
unique_users = interactions_df.user_id.nunique()
unique_items = interactions_df.item_id.nunique()

print("interactions shape: {}".format(interactions_df.shape))
print("interactions unique users: {}".format(interactions_df.user_id.nunique()))
print("interactions unique items: {}".format(interactions_df.item_id.nunique()))

print("user features users:", interactions_df.user_id.nunique())
print("item features items:", interactions_df.item_id.nunique())

interactions shape: (5930017, 3)
interactions unique users: 386390
interactions unique items: 1601277
user features users: 386390
item features items: 1601277


In [11]:
np.random.seed(1492)
interactions_df['random'] = np.random.random(size=len(interactions_df))
test_pct = -0.25#0.25

In [12]:
train_mask = interactions_df['random'] <  (1 - test_pct)
valid_mask = interactions_df['random'] >= (1 - test_pct)

interactions_train = interactions_df[train_mask][['user_id', 'item_id']]
interactions_valid = interactions_df[valid_mask][['user_id', 'item_id']]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.item_id.unique())
valid_items = np.sort(interactions_valid.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

train shape: (5930017, 2)
valid shape: (0, 2)
train users: 386390
valid users: 0
cold-start users: set()
train items: 1601277
valid items: 0
cold-start items: set()


In [13]:
user_features = users_df[users_df.user_id.isin(train_users)]
item_features = items_df[items_df.item_id.isin(train_items)]
user_features.shape, item_features.shape

((386390, 1), (1601277, 16))

In [14]:
items_df.nunique()

item_id          2102277
price             120768
condition              2
domain_bit_0           2
domain_bit_1           2
domain_bit_2           2
domain_bit_3           2
domain_bit_4           2
domain_bit_5           2
domain_bit_6           2
domain_bit_7           2
domain_bit_8           2
domain_bit_9           2
domain_bit_10          2
domain_bit_11          2
domain_bit_12          2
dtype: int64

In [15]:
model = RankFM(factors=20, loss='warp', max_samples=20, alpha=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='invscaling')
model 

In [16]:
%%time
model.fit(interactions_train, epochs=20, verbose=True)


training epoch: 0
log likelihood: -3329431.75

training epoch: 1
log likelihood: -2978897.75

training epoch: 2
log likelihood: -2853444.0

training epoch: 3
log likelihood: -2743361.25

training epoch: 4
log likelihood: -2631774.25

training epoch: 5
log likelihood: -2516611.5

training epoch: 6
log likelihood: -2403552.0

training epoch: 7
log likelihood: -2295890.75

training epoch: 8
log likelihood: -2197335.5

training epoch: 9
log likelihood: -2108425.5

training epoch: 10
log likelihood: -2025368.0

training epoch: 11
log likelihood: -1950687.5

training epoch: 12
log likelihood: -1883282.5

training epoch: 13
log likelihood: -1823335.375

training epoch: 14
log likelihood: -1768415.5

training epoch: 15
log likelihood: -1718721.625

training epoch: 16
log likelihood: -1672402.375

training epoch: 17
log likelihood: -1630756.25

training epoch: 18
log likelihood: -1591646.0

training epoch: 19
log likelihood: -1555563.625
CPU times: user 6min 22s, sys: 10.7 s, total: 6min 32s
W

In [17]:
def get_candidates(user):
    df = interactions_df[(interactions_df['user_id'] == user)]# & (interactions['event_type'] != 'search')]
    items_interacted = set(df['item_id'])
    domains = set(items_dict[int(item)]['domain_id'] for item in items_interacted)
    return [str(x) for x in sum([domain_item_dict[d] for d in domains], [])]

In [18]:
interactions[(interactions['user_id'] == 1) & (interactions['event_type'] != 'search')]

,user_id,item_id,event_type,event_timestamp,target
19,1,643652,view,2019-10-06T18:02:53.893-0400,228737
21,1,1156086,view,2019-10-07T09:46:07.960-0400,228737
24,1,1943604,view,2019-10-07T09:47:53.958-0400,228737
26,1,206667,view,2019-10-07T18:53:26.670-0400,228737
28,1,1282813,view,2019-10-07T18:54:50.998-0400,228737
29,1,228737,view,2019-10-07T18:56:43.678-0400,228737
30,1,228737,view,2019-10-07T19:01:44.718-0400,228737
31,1,228737,view,2019-10-07T19:46:18.382-0400,228737


### Create item users pairs to feed the model

In [19]:
%%time
users = train_users[:5000]
users_column = []
items_column = []
user_lengths = []
for u in users:
    candidates = get_candidates(u)
    items_column += candidates
    users_column += [u] * len(candidates)
    user_lengths.append((u, len(candidates)))
pairs = pd.DataFrame({'user_id': users_column, 'item_id': items_column})
pairs

CPU times: user 1min 38s, sys: 31.9 s, total: 2min 9s
Wall time: 2min 18s


,user_id,item_id
0,0,2061169
1,0,1110112
2,0,1428064
3,0,1779769
4,0,18680
...,...,...
97315151,5362,226463
97315152,5362,1950736
97315153,5362,135043
97315154,5362,881616


In [20]:
%%time
recommendations_pairs = model.predict(pairs, cold_start='nan')

CPU times: user 4min 33s, sys: 54.9 s, total: 5min 28s
Wall time: 19min 48s


In [21]:
print(len(recommendations_pairs))
recommendations = {}

97315156


In [22]:
%%time
offset = 0
for user, user_len in user_lengths:
    user_recs = recommendations_pairs[offset:offset+user_len]
    ranked_recs = np.argsort(user_recs)[::-1]
    top_10 = [x for x in ranked_recs if not np.isnan(user_recs[x])][:10]
    #print(top_10)
    #print([user_recs[x] for x in top_10])
    recommendations[user] = [items_column[x + offset] for x in top_10]
    offset += user_len

CPU times: user 3min 1s, sys: 6.02 s, total: 3min 7s
Wall time: 3min 24s


In [23]:
#print(top_10)

In [24]:
#items_dict[228737]

In [25]:
recommendations

{0: ['1361154',
  '1748830',
  '1647194',
  '87559',
  '1615991',
  '1154325',
  '455802',
  '400496',
  '1786148',
  '1839144'],
 1: ['1172716',
  '868817',
  '228737',
  '1817390',
  '1341158',
  '894119',
  '1165777',
  '1609425',
  '864577',
  '855886'],
 2: ['671193',
  '402169',
  '1997523',
  '941561',
  '182725',
  '1352406',
  '236078',
  '1966546',
  '1748343',
  '303917'],
 3: ['505541',
  '1498722',
  '12823',
  '1853737',
  '1377441',
  '144250',
  '878821',
  '955759',
  '446840',
  '1271307'],
 4: ['1768844',
  '1920677',
  '1707797',
  '1313192',
  '928548',
  '1926672',
  '129029',
  '1329338',
  '536540',
  '588326'],
 5: ['1135701',
  '1283928',
  '1238121',
  '1316904',
  '1214472',
  '1293013',
  '1818994',
  '1541412',
  '2029385',
  '1488196'],
 6: ['882697',
  '968229',
  '119703',
  '790888',
  '1595373',
  '722653',
  '967194',
  '1219704',
  '1787077',
  '1332849'],
 7: ['383570',
  '1662546',
  '304747',
  '1960072',
  '1851516',
  '1855537',
  '42432',
  '1

## NDCG

In [26]:
user_target_dict = interactions.groupby('user_id')['target'].unique().apply(lambda x: x).to_dict()
user_target_dict

{0: array([1748830]),
 1: array([228737]),
 2: array([1909110]),
 3: array([1197370]),
 4: array([2049207]),
 5: array([1046119]),
 6: array([1206282]),
 7: array([1393318]),
 8: array([2056361]),
 9: array([1916890]),
 10: array([392483]),
 11: array([107670]),
 12: array([537473]),
 13: array([673499]),
 14: array([65317]),
 15: array([1371364]),
 16: array([1877560]),
 17: array([232]),
 18: array([1228971]),
 19: array([1349384]),
 20: array([1752865]),
 21: array([745652]),
 22: array([1075129]),
 23: array([1993308]),
 24: array([830693]),
 25: array([1813208]),
 26: array([1567189]),
 27: array([1449644]),
 28: array([1587395]),
 29: array([1335130]),
 30: array([1882157]),
 31: array([1649032]),
 32: array([871796]),
 33: array([1310157]),
 34: array([887130]),
 35: array([1402766]),
 36: array([993444]),
 37: array([883507]),
 38: array([683871]),
 39: array([1499828]),
 40: array([1512116]),
 41: array([162734]),
 42: array([1750403]),
 43: array([1805298]),
 44: array([19191

In [27]:
def relevance(item, target):
    if item == target:
        return 12
    elif items_dict[item]['domain_id'] == items_dict[target]['domain_id']:
        return 1
    return 0

def ndcg(r, target):
    def get_perfect():
        perfect = [12, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        assert len(perfect) == 10
        return sum(perfect[i] / math.log(i + 2) for i in range(len(perfect)))
    
    dcg = sum(relevance(r[i], target) / math.log(i + 2) for i in range(len(r)))
    return dcg / get_perfect()

In [28]:
sum_ndcg = 0
for x in recommendations.keys():
    sum_ndcg += ndcg([int(w) for w in recommendations[x]], int(user_target_dict[x][0]))

print(sum_ndcg / len(recommendations.keys()))

0.2278788841265634


In [ ]:
#factors = 50 => 0.26399649718445173